# IBM Instance Setup

In [1]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [2]:
NODES = 8
SP = "Sparse"
MD = "Medium"
DS = "Dense"

ER = "ER"
BP = "Bipartite"

In [3]:
ER8NSParams = np.load("results/params/params_8N_ER_Sparse_LR0.04522709518095061_SIGMA0.6587519702447917_INIT0.17944082088572766_NUMLAYERS1.npy")
ER8NMParams = np.load("results/params/params_8N_ER_Medium_LR0.011220685773844122_SIGMA0.6962179567887462_INIT1.810103830663358_NUMLAYERS5.npy")
ER8NDParams = np.load("results/params/params_8N_ER_Dense_LR0.04206675941389984_SIGMA0.7206244967648459_INIT0.5521825859396434_NUMLAYERS1.npy")
BP8NSParams = np.load("results/params/HPO_FINAL/8N/8N_Bipartite_S_LR5.9E-02_S0.1006_I1.9967_L4.npy")
BP8NMParams = np.load("results/params/HPO_FINAL/8N/8N_Bipartite_M_LR1.4E-02_S0.16_I1.81_L3.npy")
BP8NDParams = np.load("results/params/HPO_FINAL/8N/8N_Bipartite_D_LR5.7E-03_S1.08_I1.28_L3.npy")

## Experiments

In [4]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()
dev = qml.device("lightning.qubit", 
                    wires=QUBITS, 
                    shots=512)

L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [5]:


BP8NDRes = L3Sample(BP8NDParams)
BP8NMRes = L3Sample(BP8NMParams)
BP8NSRes = L4Sample(BP8NSParams)

In [6]:
#np.save("ER8NS_simulated.npy", ER8NSRes)
#np.save("ER8NM_simulated.npy", ER8NMRes)
#np.save("ER8ND_simulated.npy", ER8NDRes)
np.save("results/samples/2025-07-03_PLQ/8N_Bipartite_S_4L_ideal_raw.npy", BP8NSRes)
np.save("results/samples/2025-07-03_PLQ/8N_Bipartite_M_3L_ideal_raw.npy", BP8NMRes)
np.save("results/samples/2025-07-03_PLQ/8N_Bipartite_D_3L_ideal_raw.npy", BP8NDRes)
print("Samples saved successfully.")

Samples saved successfully.
